# Multi-Objective  Optimization

In [ ]:
# This code requires an extension of the main 'octis' library available on GitHub.
# You can download the extension from this branch:  https://github.com/MIND-Lab/OCTIS/tree/develop-multiobjective-opt

#This library also needs to be installed:
##!pip install "git+https://github.com/MIND-Lab/dragonfly#egg=drag  onfly-opt"

In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
from octis.dataset.dataset import Dataset
from skopt.space.space import Real, Categorical, Integer
from octis.optimization.mo_optimizer import MOOptimizer

from octis.models.ETM import ETM
from octis.models.LDA import LDA
from octis.models.ProdLDA import ProdLDA

from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity 

In [ ]:
path= r"C:\myDatapath\\" #preprocessed data

In [ ]:
#importa data
dataset = Dataset()
dataset.load_custom_dataset_from_folder(path)

In [ ]:
# Instantiating the models
model = ETM(dataset, use_partitions=False, train_embeddings=False, embedding_size=100, rho_size=100, num_epochs=50,
            embeddings_type='word2vec', embeddings_path=path+'allLASLAlemmi-vector-100-nocase-w5-CBOW.vec',
            binary_embeddings=False, headerless_embeddings=False)

#LDA model
#model= LDA(dataset) #hyperparameters defined in the configuration file

#ProdLDA model
#model= ProdLDA(dataset, solver='adam', num_epochs= 100)


In [ ]:
#metrics
#deafult on topk=10
td = TopicDiversity()
coh = Coherence(texts=dataset.get_corpus(), #list of our documents
                 measure='c_npmi')
metrics_names = ['td', 'npmi', ]
name2metric = {'td':td, 'npmi':coh}
metrics = [name2metric[m] for m in metrics_names]

In [ ]:
m = 'etm' #'lda','prodlda'
d = 'sentence-based' #'work-based'


#!!!
# For each model, we need to define the hyperparameter configuration.
# The script expects to find the configuration files in a folder with this path: .\data\hyperparameter_search\.

mob = MOOptimizer(dataset=dataset, model=model, config_file= path + "/data/hyperparameter_search//" + m +"_config_topics.json", 
                  model_runs=30, number_of_calls=30, metrics=metrics,  maximize=True, acq_func='ucb', 
                  progress_file=path + '/results//' + m + '_topics' + d + '_' + '_'.join(metrics_names) + '.p',
                  save_path= path + '/results//' + m + '_topics' + d + '_' + '_'.join(metrics_names) + '//',)

In [ ]:
#optimization
mob.optimize()

#the results will be stored in a pickle file parameterized by the parameter "save_path"